<a href="https://colab.research.google.com/github/PriyaDeshpande1605/cnn-svm/blob/master/CNN_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(7*7*32, 10)
        self.fc2 = nn.Linear(10,1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.MultiLabelMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate)  # Our optimizer

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        #print(images.shape)
        outputs = model(images)
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs.reshape(100,), labels)
        optimizer.zero_grad()
        #loss = torch.mean(torch.clamp(1 - labels*outputs , min=0))  # hinge loss
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 103.0510
Epoch [1/5], Step [200/600], Loss: 81.9289
Epoch [1/5], Step [300/600], Loss: 89.6936
Epoch [1/5], Step [400/600], Loss: 99.7764
Epoch [1/5], Step [500/600], Loss: 89.6107
Epoch [1/5], Step [600/600], Loss: 94.4552
Epoch [2/5], Step [100/600], Loss: 88.8933
Epoch [2/5], Step [200/600], Loss: 80.5577
Epoch [2/5], Step [300/600], Loss: 86.9382
Epoch [2/5], Step [400/600], Loss: 91.7578
Epoch [2/5], Step [500/600], Loss: 85.5212
Epoch [2/5], Step [600/600], Loss: 92.1442
Epoch [3/5], Step [100/600], Loss: 92.6306
Epoch [3/5], Step [200/600], Loss: 84.1594
Epoch [3/5], Step [300/600], Loss: 90.2667
Epoch [3/5], Step [400/600], Loss: 82.3628
Epoch [3/5], Step [500/600], Loss: 92.9300
Epoch [3/5], Step [600/600], Loss: 87.1265
Epoch [4/5], Step [100/600], Loss: 82.3988
Epoch [4/5], Step [200/600], Loss: 105.2107
Epoch [4/5], Step [300/600], Loss: 82.7210
Epoch [4/5], Step [400/600], Loss: 89.5272
Epoch [4/5], Step [500/600], Loss: 80.6887
Epoch [4/